In [165]:
%%pyspark
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
from pyspark.ml.evaluation import RegressionEvaluator


StatementMeta(projetpfe, 0, 166, Finished, Available, Finished)

In [158]:
df = spark.read.load('abfss://csv@storagediabetes.dfs.core.windows.net/product_final.csv', format='csv', Header = True)
geog = spark.read.load('abfss://csv@storagediabetes.dfs.core.windows.net/list_geolocation_dataset.csv', format='csv'
## If header exists uncomment line below
##, header=True
)
display(df.limit(10))

StatementMeta(projetpfe, 0, 159, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6dca8b26-f04a-47e9-bbeb-20ef6d878721)

In [ ]:
df.na.drop()

In [159]:
df = df.withColumn("price", df["price"].cast(DoubleType())) \
       .withColumn("product_name_lenght", df["product_name_lenght"].cast(IntegerType())) \
       .withColumn("product_description_lenght", df["product_description_lenght"].cast(IntegerType())) \
       .withColumn("product_photos_qty", df["product_photos_qty"].cast(IntegerType())) \
       .withColumn("product_weight_g", df["product_weight_g"].cast(IntegerType())) \
       .withColumn("product_length_cm", df["product_length_cm"].cast(IntegerType())) \
       .withColumn("product_height_cm", df["product_height_cm"].cast(IntegerType())) \
       .withColumn("product_width_cm", df["product_width_cm"].cast(IntegerType())) \
       .withColumn("review_score", df["review_score"].cast(IntegerType())) 

StatementMeta(projetpfe, 0, 160, Finished, Available, Finished)

In [166]:
df.na.drop()

StatementMeta(projetpfe, 0, 167, Finished, Available, Finished)

AttributeError: 'DataFrame' object has no attribute 'apply'

df.printSchema()

df.describe().show()

Convertir les catégories de produits en variables numériques

In [133]:
indexer = StringIndexer(inputCol="product_category_name", outputCol="product_category_index")
df = indexer.fit(df).transform(df)
df.show(5)

StatementMeta(projetpfe, 0, 134, Finished, Available, Finished)

+--------------------+-----+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+----------------------+
|          product_id|price|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|review_score|product_category_index|
+--------------------+-----+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+------------+----------------------+
|4244733e06e7ecb49...| 58.9|           cool_stuff|                 58|                       598|                 4|             650|               28|                9|              14|           5|                  11.0|
|e5f2d52b802189ee6...|239.9|             pet_shop|                 56|                       239|           

In [134]:
feature_columns = ['price', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'product_category_index']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

StatementMeta(projetpfe, 0, 135, Finished, Available, Finished)

In [135]:
df = df.withColumn("product_category_index", df["product_category_index"].cast(IntegerType()))

StatementMeta(projetpfe, 0, 136, Finished, Available, Finished)

In [136]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features", handleInvalid="skip")
df = assembler.transform(df)

StatementMeta(projetpfe, 0, 137, Finished, Available, Finished)

In [137]:
df = df.select(['features', 'review_score'])

StatementMeta(projetpfe, 0, 138, Finished, Available, Finished)

In [138]:
df.na.drop()

StatementMeta(projetpfe, 0, 139, Finished, Available, Finished)

DataFrame[features: vector, review_score: int]

In [139]:
df.printSchema()

StatementMeta(projetpfe, 0, 140, Finished, Available, Finished)

root
 |-- features: vector (nullable = true)
 |-- review_score: integer (nullable = true)



In [151]:
df.fillna(0)

StatementMeta(projetpfe, 0, 152, Finished, Available, Finished)

DataFrame[features: vector, review_score: int]

In [140]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

StatementMeta(projetpfe, 0, 141, Finished, Available, Finished)

In [141]:
lr = LinearRegression(labelCol="review_score")
lr_model = lr.fit(train_df)

StatementMeta(projetpfe, 0, 142, Finished, Available, Finished)

In [142]:
predictions = lr_model.transform(test_df)

StatementMeta(projetpfe, 0, 143, Finished, Available, Finished)

In [143]:
predictions.select("prediction", "review_score", "features").show()

StatementMeta(projetpfe, 0, 144, Finished, Available, Finished)

+------------------+------------+--------------------+
|        prediction|review_score|            features|
+------------------+------------+--------------------+
| 4.078981169881583|           5|[2.9,48.0,1392.0,...|
| 4.041787129784572|           5|[3.85,52.0,446.0,...|
| 4.129007477861675|           5|[3.9,39.0,275.0,2...|
| 4.129007477861675|           5|[3.9,39.0,275.0,2...|
| 4.051580126112524|           1|[4.99,43.0,454.0,...|
| 4.013595207318975|           5|[4.99,47.0,313.0,...|
| 4.057355860338315|           4|[5.2,40.0,781.0,3...|
| 4.099494803594185|           5|[5.49,23.0,3132.0...|
| 4.016037792332077|           3|[5.85,28.0,834.0,...|
|4.0234805527871575|           3|[5.85,54.0,458.0,...|
|4.0234805527871575|           3|[5.85,54.0,458.0,...|
| 4.040925413242543|           5|[5.9,22.0,332.0,1...|
| 3.986188671948315|           1|[5.9,41.0,400.0,1...|
| 3.986188671948315|           1|[5.9,41.0,400.0,1...|
|  4.12305025602049|           4|[5.9,60.0,981.0,1...|
| 4.120135

In [144]:
evaluator = RegressionEvaluator(labelCol="review_score", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print(f"Mean Absolute Error (MAE): {mae}")

StatementMeta(projetpfe, 0, 145, Finished, Available, Finished)

Mean Absolute Error (MAE): 1.1129464116769296


In [145]:
evaluator = RegressionEvaluator(labelCol="review_score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

StatementMeta(projetpfe, 0, 146, Finished, Available, Finished)

Root Mean Squared Error (RMSE): 1.4043783144193012


In [154]:
lr_model.save("home/thomas.tisseron/lr_model")

StatementMeta(projetpfe, 0, 155, Finished, Available, Finished)

In [156]:
loaded_model = LinearRegressionModel.load("home/thomas.tisseron/lr_model")

StatementMeta(projetpfe, 0, 157, Finished, Available, Finished)

In [157]:
new_predictions = loaded_model.transform(test_df)
new_predictions.select("prediction", "review_score", "features").show()

StatementMeta(projetpfe, 0, 158, Finished, Available, Finished)

+------------------+------------+--------------------+
|        prediction|review_score|            features|
+------------------+------------+--------------------+
| 4.078981169881583|           5|[2.9,48.0,1392.0,...|
| 4.041787129784572|           5|[3.85,52.0,446.0,...|
| 4.129007477861675|           5|[3.9,39.0,275.0,2...|
| 4.129007477861675|           5|[3.9,39.0,275.0,2...|
| 4.051580126112524|           1|[4.99,43.0,454.0,...|
| 4.013595207318975|           5|[4.99,47.0,313.0,...|
| 4.057355860338315|           4|[5.2,40.0,781.0,3...|
| 4.099494803594185|           5|[5.49,23.0,3132.0...|
| 4.016037792332077|           3|[5.85,28.0,834.0,...|
|4.0234805527871575|           3|[5.85,54.0,458.0,...|
|4.0234805527871575|           3|[5.85,54.0,458.0,...|
| 4.040925413242543|           5|[5.9,22.0,332.0,1...|
| 3.986188671948315|           1|[5.9,41.0,400.0,1...|
| 3.986188671948315|           1|[5.9,41.0,400.0,1...|
|  4.12305025602049|           4|[5.9,60.0,981.0,1...|
| 4.120135